# Notebook 3: Handling DICOM format

Images stored in the DICOM format have a meta-data dictionary associated with them, which is populated with the DICOM tags. 

When a DICOM series is read as a single image, the meta-data information is not available since DICOM tags are specific to each file. 

If you need the meta-data, you have three options:

1. Using the object oriented interface's [ImageSeriesReader](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1ImageSeriesReader.html) class, configure it to load the tags using the `MetaDataDictionaryArrayUpdateOn` method and possibly the `LoadPrivateTagsOn` method if you need the private tags. Once the series is read you can access the meta-data from the series reader using the `GetMetaDataKeys`, `HasMetaDataKey`, and `GetMetaData`.

2. Using the object oriented interface's [ImageFileReader](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1ImageFileReader.html), set a specific slice's file name and only read it's meta-data using the `ReadImageInformation` method which only reads the meta-data but not the bulk pixel information. Once the meta-data is read you can access it from the file reader using the `GetMetaDataKeys`, `HasMetaDataKey`, and `GetMetaData`.

3. Using the object oriented interface's [ImageFileReader](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1ImageFileReader.html), set a specific slice's file name and read it. Or using the procedural interface's, [ReadImage](https://itk.org/SimpleITKDoxygen/html/namespaceitk_1_1simple.html#ae3b678b5b043c5a8c93aa616d5ee574c) function, read a specific file. You can then access the meta-data directly from the [Image](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1Image.html) using the `GetMetaDataKeys`, `HasMetaDataKey`, and `GetMetaData`.

In [0]:
!git clone https://github.com/albertine/RPM_IBM_Module_IA.git

## 1. First method to load DICOM images
In the following cells, we read DICOM series, display them and write them as a single file (mha or nii)

In [0]:
# Import required packages
import numpy as np
import matplotlib.pyplot as plt
import os

# Install and import SimpleITK
!pip install --upgrade SimpleITK
import SimpleITK as sitk

# Import some interactive IPython widgets
# The interact function (ipywidgets.interact) automatically creates user interface (UI) controls for exploring code and data interactively.
from ipywidgets import interact, interactive, fixed

# Define current working directory
PWD_DIR = os.getcwd()
# Define output data directory
OUTPUT_DIR = os.path.join(PWD_DIR,'output')
# If it doesn't exist, create it
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [0]:
# Define DICOM data directory
#data_dcm_directory = '/content/drive/My Drive/data/patient2/'
data_dcm_directory = '/content/RPM_IBM_Module_IA/data/patient2/'
print("Path to DICOM data directory:",data_dcm_directory)

# Provide a file name that belongs to the DICOM CT series you want to read
!ls '/content/RPM_IBM_Module_IA/data/patient2/'
file_name_ct='/content/RPM_IBM_Module_IA/data/patient2/Patient2.CT._.4.1.2019.11.21.11.52.34.596.71223801.dcm'

# Read the file's meta-information without reading bulk pixel data
file_reader_ct=sitk.ImageFileReader()
file_reader_ct.SetFileName(file_name_ct)
file_reader_ct.ReadImageInformation()

# Get the sorted file names, open all files in the directory and read the meta-information without reading the bulk pixel data
series_ID_ct=file_reader_ct.GetMetaData('0020|000e')
sorted_file_names_ct=sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_dcm_directory,series_ID_ct)
print("Series ID corresponding to CT data:",series_ID_ct)

# Provide a file name that belongs to the DICOM PET series you want to read
file_name_pet='/content/RPM_IBM_Module_IA/data/patient2/Patient2.PT._.3.144.2019.11.21.11.52.34.596.71220765.dcm'
# Read the file's meta-information without reading bulk pixel data
file_reader_pet=sitk.ImageFileReader()
file_reader_pet.SetFileName(file_name_pet)
file_reader_pet.ReadImageInformation()

# Get the sorted file names, open all files in teh directory and read the meta-information without reading the bulk pixel data
series_ID_pet=file_reader_pet.GetMetaData('0020|000e')
sorted_file_names_pet=sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_dcm_directory,series_ID_pet)
print("Series ID corresponding to PET data:",series_ID_pet)

In [0]:
# Read DICOM CT and PET images and convert them to NumPy arrays
# Access to their attributes (size, origin and spacing)
img_ct=sitk.ReadImage(sorted_file_names_ct)
nimg_ct = sitk.GetArrayFromImage(img_ct)
print('Array CT size: ', nimg_ct.shape)
print('Image CT origin: ', img_ct.GetOrigin())
print('Image CT spacing: ', img_ct.GetSpacing(),'\n')

img_pet=sitk.ReadImage(sorted_file_names_pet)
nimg_pet = sitk.GetArrayFromImage(img_pet)
print('Array PET size: ', nimg_pet.shape)
print('Image PET origin: ', img_pet.GetOrigin())
print('Image PET spacing: ', img_pet.GetSpacing())

In [0]:
# Read the bulk pixel data and write them as a single 3D nii or mha file
output_file_name_3D_ct = os.path.join(OUTPUT_DIR, '3DImageCT.nii') #'3DImageCT.mha'
sitk.WriteImage(img_ct, output_file_name_3D_ct)
output_file_name_3D_pet = os.path.join(OUTPUT_DIR, '3DImagePET.nii') #'3DImagePET.mha'
sitk.WriteImage(img_pet, output_file_name_3D_pet)

## 2. DICOM image display

### Using matplotlib and interact function

By converting into a NumPy array, matplotlib can be used for visualization for integration into the scientific Python environment. 

This is good for illustrative purposes, but is problematic when working with images that have a high dynamic range or non-isotropic spacing - most 3D medical images.

In [0]:
# Home-made function to show a 3D image in three views
def show_image(arr,sx,sy,sz):
    fig, ax = plt.subplots(1,3, figsize=(10, 10))
    ax[0].imshow(arr[sz,:,:], cmap=plt.cm.gray)
    ax[1].imshow(arr[:,sx,:], cmap=plt.cm.gray, origin='lower')
    ax[2].imshow(arr[:,:,sy], cmap=plt.cm.gray, origin='lower')

In [0]:
# View CT
interact(show_image, arr=fixed(nimg_ct), sx=(0,img_ct.GetSize()[0]), sy=(0,img_ct.GetSize()[2]), sz=(0,img_ct.GetSize()[1]));

In [0]:
# View PET
interact(show_image, arr=fixed(nimg_pet), sx=(0,img_pet.GetSize()[0]), sy=(0,img_pet.GetSize()[1]), sz=(0,img_pet.GetSize()[2]));

In [0]:
# Home-made function to overlay two 3D images from different modality imaging (here, CT and PET)
def show_fusion(arr, nslice, opacity=0.7 ):
    plt.imshow(nimg_pet[nslice,:,:], cmap=plt.cm.hot)
    plt.colorbar()
    a = arr[nslice,:,:]
    plt.imshow(a, alpha=opacity, cmap=plt.cm.gray)
    plt.colorbar()

In [0]:
# View overlaid CT and PET images
interact(show_fusion, arr=fixed(nimg_ct), nslice=(0,len(nimg_pet)), opacity=(0,1,0.1));

In [0]:
# Assuming these two images are already registered, we can resample the CT volume according to the origin, spacing and dimensions of the PET volume.
# Hence, we preserve their individual locations as each has a different origin, spacing and orientation in the same space.
img_resampled_ct = sitk.Resample(img_ct, img_pet, sitk.Transform(), sitk.sitkLinear, 0)
nimg_resampled_ct = sitk.GetArrayFromImage(img_resampled_ct)
output_file_name_3D_resampled_ct = os.path.join(OUTPUT_DIR, '3DImageResampledCT.nii')
sitk.WriteImage(img_resampled_ct, output_file_name_3D_resampled_ct)

In [0]:
# View overlaid CT and PET images after having resampled CT
interact(show_fusion, arr=fixed(nimg_resampled_ct), nslice=(0,len(nimg_pet)), opacity=(0,1,0.1));

### Using SimpleITK

When working with medical images it is recommended to visualize them using dedicated software such as the freely available 3D Slicer or ITK-SNAP.

While SimpleITK does not do visualization, it does contain a built in <code>Show</code> method. This function writes the image out to disk and than launches a program for visualization. By default it is configured to use ImageJ, because it readily supports many medical image formats and loads quickly. However, the Show visualization program is easily customizable by an environment variable:

* <code>SITK_SHOW_COMMAND</code>: Viewer to use (ITK-SNAP, 3D Slicer...)
* <code>SITK_SHOW_COLOR_COMMAND</code>: Viewer to use when displaying color images.
* <code>SITK_SHOW_3D_COMMAND</code>: Viewer to use for 3D images.


So if you really want to look at your images, use the <code>sitk.Show</code> command:

In [0]:
sitk.Show?

In [0]:
%env SITK_SHOW_COMMAND "/volatile/home/DUBOIS/Info/Logiciels/Fiji.app/ImageJ-linux64"

try:
    sitk.Show(img_pet)
except RuntimeError:
    print('SimpleITK Show method could not find the viewer (ImageJ not installed or ' +
          'environment variable pointing to non existant viewer).')

Use a different viewer by setting environment variable(s). Do this from within your Jupyter notebook using 'magic' functions, or set in a more permanent manner using your OS specific convention.

In [0]:
%env SITK_SHOW_COMMAND "/volatile/home/DUBOIS/Info/Logiciels/itksnap-3.6.0-20170401-Linux-x86_64/bin/itksnap"

pet_image = sitk.ReadImage(output_file_name_3D_pet)
try:
    sitk.Show(pet_image)
except RuntimeError:
    print('SimpleITK Show method could not find the viewer (ITK-SNAP not installed or ' +
          'environment variable pointing to non existant viewer).')

In [0]:
%env SITK_SHOW_COMMAND "/volatile/home/DUBOIS/Info/Logiciels/Slicer-4.10.2-linux-amd64/Slicer"

pet_image = sitk.ReadImage(output_file_name_3D_pet)
try:
    sitk.Show(pet_image)
except RuntimeError:
    print('SimpleITK Show method could not find the viewer (Slicer not installed or ' +
          'environment variable pointing to non existant viewer).')

## 3. Second method to load DICOM images
Now, we select a specific DICOM series from a directory and only then load our selection

In [0]:
data_dcm_directory = '/content/RPM_IBM_Module_IA/data/patient2/'
print("Path to DICOM data directory:",data_dcm_directory)

# Global variable 'selected_series' is updated by the interact function
selected_series = ''
file_reader = sitk.ImageFileReader()
def DICOM_series_dropdown_callback(series_to_load, series_dictionary):
    global selected_series
               # Print some information about the series from the meta-data dictionary
               # DICOM standard part 6, Data Dictionary: http://medical.nema.org/medical/dicom/current/output/pdf/part06.pdf
    file_reader.SetFileName(series_dictionary[series_to_load][0])
    file_reader.ReadImageInformation()
    tags_to_print = {'0010|0010': 'Patient name: ', 
                     '0008|0060' : 'Modality: ',
                     '0008|0021' : 'Series date: ',
                     '0008|0080' : 'Institution name: '}
    for tag in tags_to_print:
        try:
            print(tags_to_print[tag] + file_reader.GetMetaData(tag))
        except: # Ignore if the tag isn't in the dictionary
            pass
    selected_series = series_to_load

    
# Directory contains multiple DICOM studies/series, store
             # in dictionary with key being the series ID
reader = sitk.ImageSeriesReader()
series_file_names = {}  
series_IDs = reader.GetGDCMSeriesIDs(data_dcm_directory)

In [0]:
# Display both modality imaging side by side
cpt=0
# Check that we have at least one series
if series_IDs:
    for series in series_IDs:
        print('Series ID:', series)
        series_file_names[series] = reader.GetGDCMSeriesFileNames(data_dcm_directory, series)
        interactive(DICOM_series_dropdown_callback, series_to_load=series, series_dictionary=fixed(series_file_names));
        reader.SetFileNames(series_file_names[series])
        reader.MetaDataDictionaryArrayUpdateOn()
        reader.LoadPrivateTagsOn()
        img = reader.Execute()
        modality=reader.GetMetaData(2,'0008|0060')
        print('Modality:',modality)
        # Display the image slice from the middle of the stack, z axis
        z = int(img.GetDepth()/2)
        plt.subplot(1,2,cpt+1)
        plt.imshow(sitk.GetArrayViewFromImage(img)[z,:,:], cmap=plt.cm.Greys_r)
        plt.title('Modality: ' + modality)
        plt.axis('off');
        cpt+=1
else:
    print('Data directory does not contain any DICOM series.')